In [1]:
# simplest data to work with : tiny shakespare 
with open('input.txt', 'r') as f:
    text = f.read()
data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens)

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13, 198, 198, 5962, 22307, 25, 198, 1639, 389, 477, 12939, 2138, 284, 4656, 621, 284, 1145, 680, 30, 198, 198, 3237, 25, 198, 4965, 5634, 13, 12939, 13, 198, 198, 5962, 22307, 25, 198, 5962, 11, 345, 760, 327, 1872, 385, 1526, 28599, 318, 4039, 4472, 284, 262, 661, 13, 198, 198, 3237, 25, 198, 1135, 760, 470, 11, 356, 760, 470, 13, 198, 198, 5962, 22307, 25, 198, 5756, 514, 1494, 683, 11, 290, 356, 1183, 423, 11676, 379, 674, 898, 2756, 13, 198, 3792, 470, 257, 15593, 30, 198, 198, 3237, 25, 198, 2949, 517, 3375, 319, 470, 26, 1309, 340, 307, 1760, 25, 1497, 11, 1497, 0, 198, 198, 12211, 22307, 25, 198, 3198, 1573, 11, 922, 4290, 13, 198, 198, 5962, 22307, 25, 198, 1135, 389, 17830, 3595, 4290, 11, 262, 1458, 1173, 1547, 922, 13, 198, 2061, 4934, 969, 5036, 896, 319, 561, 26958, 514, 25, 611, 484, 198, 19188, 7800, 514, 475, 262, 48713, 414, 11, 981, 340, 547, 198, 

In [5]:
# now we want to take these tokens and forward them in our GPT model 
# to do that we have reshape our long list of token sequence into a (B, T) tensor 
# before that tha shape our forward function takes
# take an example of 24 tokens to understand this

import torch
buf = torch.tensor(tokens[:24])
x = buf.view(4, 6)
x 

tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])

- So, that's our input to the transformer. Now, we also need labels to calucate the loss function. 
- For this we can write some code in the forward pass of the GPT model, as we know the next sequence is just the right of us. But the problem with that approch is that the for the token 13 (the last toke) we don't have the label information. 
- So Andrej fav way to do this, is that we will create the another tensor label of exact same size as x but it contains the labels at every position. 


In [7]:
# so, in the buf we get the last token + 1 and then index our x and y 
buf = torch.tensor(tokens[:24 + 1 ])
x = buf[:-1].view(4, 6) 
y = buf[1:].view(4, 6) 
x, y 

(tensor([[ 5962, 22307,    25,   198,  8421,   356],
         [ 5120,   597,  2252,    11,  3285,   502],
         [ 2740,    13,   198,   198,  3237,    25],
         [  198,  5248,   461,    11,  2740,    13]]),
 tensor([[22307,    25,   198,  8421,   356,  5120],
         [  597,  2252,    11,  3285,   502,  2740],
         [   13,   198,   198,  3237,    25,   198],
         [ 5248,   461,    11,  2740,    13,   198]]))

- Now we go back to our script and create a very simpel dataloader object to load these tokens, feed them to transformer and calculate the loss. 

----- 

Now, we want to calculate the loss function and to do so we have to modify the forward pass

using device: cuda
Only loading first 1k charchters as model input
logits shape torch.Size([4, 32, 50257])
loss tensor(11.0371, device='cuda:0', grad_fn=<NllLossBackward0>)

In [14]:
import math
1/ 50257, -math.log(50257)

(1.9897725689953638e-05, -10.82490511970208)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
for i in range(50):
    optimizer.zero_grad()
    logits, loss = model(x, y)
    loss.backward()
    optimizer.step()
    print(f"step {i}, loss: {loss.item()}")

# when we use this we are overfitting on the one batch x that we are passing again and again.
# lr = 3e-4 : is like good standard learning rate for early debugging stage. 


- Next, we don't want to overfit our model on that one batch. We want to create the a data loader which makes sure that we are getting a fresh batch. |

- num of batches in a single epoch of iterating over a single dataset
- So 1 epoch : how many unique batches do we output/iterate over before we loop back begining of the document and start reading it again. 
1000/ (4 * 32)

In [18]:
1000 // (4 * 32), 1115394 // (4 * 32) # 1115394: num of charachter but we have to use the num 
# of tokens
# So 

(7, 8714)

In [ ]:
# 1 epoch => num of tokens // (B, T)

In [20]:
# tiktokener have roughly about 1 to 3 compression ratio

In [19]:
# using device: cuda
# loaded 338025 tokens
# 1 epoch = 2640 batches
# step 0, loss: 11.08803939819336
# step 1, loss: 9.717626571655273
# step 2, loss: 8.71585750579834
# step 3, loss: 8.921952247619629
# step 4, loss: 8.44729995727539
# step 5, loss: 8.131488800048828
# step 6, loss: 9.005425453186035
# step 7, loss: 8.671504020690918
# step 8, loss: 8.075515747070312
# step 9, loss: 7.828042984008789
# step 10, loss: 8.26616096496582
# step 11, loss: 7.185250282287598
# step 12, loss: 7.63140869140625
# step 13, loss: 7.262790679931641
# step 14, loss: 7.483865261077881
# step 15, loss: 7.127208232879639
# step 16, loss: 7.217015743255615
# step 17, loss: 8.252432823181152
# step 18, loss: 7.073562145233154
# step 19, loss: 7.671370029449463
# step 20, loss: 7.384253025054932
# step 21, loss: 7.580746173858643
# step 22, loss: 6.214944839477539
# step 23, loss: 6.54738712310791
# step 24, loss: 6.535855293273926
# step 25, loss: 6.208362579345703
# step 26, loss: 6.298552989959717
# step 27, loss: 7.4446821212768555
# step 28, loss: 6.870301723480225
# step 29, loss: 6.647202968597412
# step 30, loss: 6.902033805847168
# step 31, loss: 6.967312335968018
# step 32, loss: 6.967711448669434
# step 33, loss: 6.67465353012085
# step 34, loss: 7.907723426818848
# step 35, loss: 7.669620513916016
# step 36, loss: 7.351398944854736
# step 37, loss: 7.470437049865723
# step 38, loss: 7.569716453552246
# step 39, loss: 7.141706943511963
# step 40, loss: 7.261336803436279
# step 41, loss: 6.442892551422119
# step 42, loss: 6.642857551574707
# step 43, loss: 6.760526180267334
# step 44, loss: 6.7352776527404785
# step 45, loss: 6.635310173034668
# step 46, loss: 5.62755012512207
# step 47, loss: 5.949457168579102
# step 48, loss: 6.788867473602295
# step 49, loss: 6.460808753967285

## Bug while loading the weight from hugging face 

In [22]:
from transformers import GPT2LMHeadModel
model_hf = GPT2LMHeadModel.from_pretrained('gpt2') 
sd_hf = model_hf.state_dict()


/home/shubham/projects/pytorch-concepts/torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
print(sd_hf['lm_head.weight'].shape)
print(sd_hf["transformer.wte.weight"].shape)

torch.Size([50257, 768])
torch.Size([50257, 768])


In [ ]:
# transformer.wte : is the token embedding at the bottom of the transformer
# lm_head : is the language modeling head at the top of the transformer
        #  this is taking the 768 channels of the transformer and up scale that to 
        #   50257 to get the logits for the next token

# both of these are 2-dim tensor and there shape is identical 

In [26]:
# comparing there every single element
(sd_hf['lm_head.weight'] == sd_hf['transformer.wte.weight']).all()
# as you can see that all the elemets are equal

tensor(True)

In [27]:
# checking the data pointer 
print(sd_hf['lm_head.weight'].data_ptr())
print(sd_hf['transformer.wte.weight'].data_ptr())

130897059688403
130897059688403


In [28]:
# as you can see they have the same pointer to data as well. 
# so what's happending here is that they have same weight are sharing b/w these two

# look the 3.4 Embedding and softmax section of attention all you need paper
# The original ideas comes from the paper: Using output embedding to Language models

# intiutive idea
# You want these two matrices to behave similar in the following sense
# if two tokens are very similar semantically like one of them is a lowercase and other 
# is all uppercase or the same token in a different langauage or something like that. 

# If you have similarity b/w two token presumbly you would expect that they are near by 
# in the token embedding space but in the exact same way you'd except that if you have two 
# tokens that are similar semantically you'd except them to get same probablities at the 
# output of the transfomer because they are semantically similar. 

# so both positions in the transfomers at the very bottom and at the top have this property 
# that similar tokens should have similar embeddings.

# They also observed (in the paper) if you look at the output embeddings they also behaves 
# like word embeddings 

# so basicalyy wte is used twice 
# 1. at the bottom of the transformer 
# 2. at the top of the transformer
# and In the backward pass we will get gradient contributions from both branches and 
# these gradients will add up on wte tensor
# so we will get contribution from the classifier layer and also at the bottom of the 
# transfomer 

# other than this this also helps in computations. as you can these are lot of 
# parameters

In [29]:
50257 * 768

38597376

In [ ]:
# rought like 40M parameters are shared 

- So that's the change we want to make in out code. is to share wte 
--- 
- after changing the code

In [ ]:
# sing device: cuda
# loaded 338025 tokens
# 1 epoch = 2640 batches
# step 0, loss: 10.969259262084961
# step 1, loss: 9.712841033935547
# step 2, loss: 9.00924015045166
# step 3, loss: 9.145369529724121
# step 4, loss: 8.583311080932617
# step 5, loss: 8.283556938171387
# step 6, loss: 8.973587989807129
# step 7, loss: 8.775681495666504
# step 8, loss: 8.148415565490723
# step 9, loss: 7.83228063583374
# step 10, loss: 8.298928260803223
# step 11, loss: 7.384541034698486
# step 12, loss: 7.6895294189453125
# step 13, loss: 7.419340133666992
# step 14, loss: 7.567596435546875
# step 15, loss: 7.4096784591674805
# step 16, loss: 7.3226213455200195
# step 17, loss: 8.266316413879395
# step 18, loss: 7.1639533042907715
# step 19, loss: 7.797882556915283
# step 20, loss: 7.549393177032471
# step 21, loss: 7.749938011169434
# step 22, loss: 6.446366786956787
# step 23, loss: 6.835708141326904
# step 24, loss: 6.883994102478027
# step 25, loss: 6.704035758972168
# step 26, loss: 6.787974834442139
# step 27, loss: 7.700334072113037
# step 28, loss: 7.19415807723999
# step 29, loss: 7.0182342529296875
# step 30, loss: 6.9842987060546875
# step 31, loss: 7.323225498199463
# step 32, loss: 7.172547340393066
# step 33, loss: 7.044727325439453
# step 34, loss: 7.917436122894287
# step 35, loss: 7.7980523109436035
# step 36, loss: 7.7756428718566895
# step 37, loss: 7.733654499053955
# step 38, loss: 8.064729690551758
# step 39, loss: 7.523628234863281
# step 40, loss: 7.474881172180176
# step 41, loss: 7.044745922088623
# step 42, loss: 7.131816387176514
# step 43, loss: 7.149776458740234
# step 44, loss: 7.066969394683838
# step 45, loss: 7.148806095123291
# step 46, loss: 6.268684387207031
# step 47, loss: 6.4054179191589355
# step 48, loss: 6.962234020233154
# step 49, loss: 6.824957847595215

----
## Fixing model Intialization
----

- the GPT2 and GPT3 paper are not clear about the intinilzation of the weighs. So we will look the GPT2 code.
- Along with the intinalzation from the code. when you look at the GPT2 paper under take 2. "we scale the weights of residual layers at initialization by a factor of 1/ sqrt(N) where N is the number of residual layers. 

In [30]:
# motivation on the second point on what mean in the paper. 
# you start with zero in the residual stream \
# if you look the code where we added the residuals 
    # def forward(self, x): of class Block
    #     x = x + self.attn(self.ln_1(x))
    #     x = x + self.mlp(self.ln_2(x))
# due to this addition the varience of these activations in 
# residual stream grows 

x = torch.zeros(768) # this is the residual stream 
n = 100 # eg: 100 layers 

for i in range(n):
    # randn: gives us the normal distribution with 
    # mean = 0 and std =1 
    x += torch.randn(768)

print(x.std())

tensor(9.9437)


In [ ]:
# and due to this addtion if you all the std() at the end 
# it grows to 10.
# and the scaling factor that they used in the paper is exactly compensates 
# for that growth. 

# Now, if we scale down every one of these contributions by 1/sqrt(N)

In [32]:
x = torch.zeros(768) # this is the residual stream 
n = 100 # eg: 100 layers 
for i in range(n):
    # randn: gives us the normal distribution with 
    # mean = 0 and std =1 
    x += n ** -0.5 * torch.randn(768)
print(x.std())

tensor(1.0184)


In [33]:
# now, if we scale in this then you can see we got 1. 

- So this is the way to control the growth of activations inside the residual stream in the forward pass. 

In [ ]:
# we want to initilizat the weight at the end of each block so the .c_proj(x) layer 
# so we go where have intinilized c_proj i.e 
# to do this we go to CausalSelfAttention --> __init__ 

        # self.c_proj.NANOGPT_SCALE_INIT = 1
#  NANOGPT_SCALE_INIT this will act like as a flag to set the scaling in std
            # if hasattr(module, 'NANOGPT_SCALE_INIT'):
            #     std *= (2 * self.config.n_layer) ** -0.5
# (what we discuss above)

--- 
#### So at this point we have intilized the GPT2 model correctly, we have created the data loader and we can train. 
Now Comes the fun part we can speed up the training by a lot
--- 


In [34]:
# to break the vscode script into an interactive ipython shell
# import code ; code.interact(local=locals())

# do that can check the dtype of the logits 
# logits.dtype >>> torch.float32

In [ ]:
# so by default in pytorch when you create the tensors, and this is the case 
# for all activation and the paramets fo the NN 
# by default everything is in float32 
# >>>> that means everything is using the float representation that has 32 bits
# (1 byte = 8 bits)

# and that's a quite a bit of memory specially for deep learning workloads 
# and deep learnig workflow can torelate small precisions but this is not true for 
# all computational workflows


# FP32 --> 19.5 TFOPS meaning 
# we means we can do 19.5 trillion operations 

# int8 is used for inference but not for training 

# In addition to that if all of these number have fewer bits of representation. 
# It's going to much easier to move them around and that's where we start to 
# get into memeory bandwidth and the memory of the model. 

# Not only we have the finite capacity of number of bits that our GPU can store (GPU memory)
# but in addition to that there is speed with which you can access this memory (Memory Bandwidth)

# And, most of the Deep learning application are memory bound. 
# our tensor are sitting idel waiting for data to come. 
# so, if you are at 60% data utilization that means we are doing it quite well. 



In [ ]:
# my spec 
# RTX 3090, 24 GB 
# CUDA Cores: 10,496
# Tensor Cores: 328
# Ray Tracing Cores: 82
# Stream Multiprocessors (SM): 82, with 128 CUDA cores per SM

---
### Looking at the Tensor Float 32 [TF32]
---

In [ ]:
# what are tensor cores???
# they are intruction 
# so everytime you have a task say matmul
# they get broken into 4x4 multiply intruction 

# so if you look at the the entire transformer is just the matix multiply
# In our network, the 124M the 768 x 50527 this operation happend in the last 
# classification layer and this dominates everything else that is happening in the NN


# FP32 is th 32 bits 
# and, TF32 is the exact same bits : (we have one sign bit and 8 exponent bit)
# but the mantissa get cropped. so basically we have 19 bits instead of 32 bits
# because the last 13 bits get dropped and all of this is internal to the instruction . 
# that means none of our pytorch code will change 

# this speed up comes at the cost that we are reducing the presion. 
# our accumulate is still an FP32, our output is FP32 , input are FP32
# but internally things get truncated to perform these operation faster. 
# But, when you train with this you can't really tell the difference. 

In [ ]:
# your CPU is just scheduling some work on GPU 

In [ ]:
# changes 
#  DataLoaderLite(B=4, T=32) to DataLoaderLite(B=16, T=1024)
#>>>> torch.OutOfMemoryError: CUDA out of memory. Tried to allocate 3.07 GiB. GPU 0 has a total capacity of 23.70 GiB of which 267.19 MiB is free. Including non-PyTorch memory, this process has 23.15 GiB memory in use. Of the allocated memory 21.75 GiB is allocated by PyTorch, and 270.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#DataLoaderLite(B=8, T=1024) > 550ms   
# memory consumption was 20.5GB on GPU

# sso this is baseline in float 32 
# using device: cuda
# loaded 338025 tokens
# 1 epoch = 41 batches
# step 49, loss: 5.919715404510498, dt: 541.97ms, tok/sec: 15115.280568754155

# Andrej suggestion 
# you want to max out the batch_size that you can fit in your GPU
# and use nice numbers: numbers that have lots of powers of two in them 
# so 16, 8, 24, 32 and so on 

# we might change the batch size but token_per_sec is  the objective that we care about. 
# token_per_sec
# how much of tokens of data are we training on and what is the throughput of 
# tokens that we're getting in our optimization 

# Andrej process : 16322 token 

In [ ]:
# check the pytorch documentation for this
torch.set_float32_matmul_precision('high')
# after setting this all the matmul inside the linear layer should utilize the 
# tensor core with TF32 precision. 

# step 49, loss: 5.917618274688721, dt: 405.04ms, tok/sec: 20225.006897509764

# we got 20225 which was 15115 
# but andrej got around 3x more token (49k per second) instead of 8x 
# reason: a lot of these computation are memory bound. All the number are still float32 
# they are being shipped all over the system and costing us way too much time to shuffle 
# around the data. So even thought we have made the multiply much faster, we are memory 
# bound and we are not getting the full benefit

# try updating the cuda version 

----
FLOAT16  (All of this we are reading from Nvidia A100 GPU)

----

https://developer.nvidia.com/blog/nvidia-ampere-architecture-in-depth/
FIG 7 

- Maitaining 16bits per float
- exponent part of the float representation: sets the range that you can represent of your numbers and the Precision is how much precision you have for your numbers. 
- so, the FP16 can't represent the full range of FP32 it has a reduced range and that's where you start to run into issue because now you have these gradient scalers.
- So FP16 came in volta series before amphere and everyone in the AI space started to train FP16 but they had you use all these gradient scaling operations which are annoying and they additional source of state of complexity.And, the reason for that 
exponent range is reduced in FP16. 
- And then they came out with BF16 in the Amphere and they made it much simplier because we are just truncating mantissa and we have exact same range and we don't need the gradeint scaler. 
- Using BF16 might be impacting the number that we see our pytorch code. 
- Look at https://pytorch.org/tutorials/recipes/recipes/amp_recipe.html
- Ignore everything and only look at torch.autocast 
- if you look at document of torch.autocat it says that 
    - you only need to run your model and loss under autocast context manager
    - see for context  https://pytorch.org/docs/stable/amp.html#torch.autocast

In [35]:
    # with torch.autocast(device_type=device, dtype=torch.bfloat16):
    #     logits, loss = model(x, y)  

# this is only change we have to do in our code. 
# In contrast with FP32 this is going to impact our tensors 

# if you look at the logits.dtype >> torch.bfloat16 
# mode.transformer.wte.weight.dtype >> torch.float32

# so our parameters are still float32 but the activation (logits) are bfloat16
# so this is why it's called a Mixed-precision. 

# Somethings in pytorch are convert to lower precision and somethings are kept to float32

# and what gets converted is not very clear in the pytorch documentation. 

# so with BFLOAT16 are are losing the precision but it's a good trade as we can train faster. 
# since, we can train faster we can train our model for more iteration and over come that trade off.

In [ ]:
# ** Mixed precision 
# ( DataLoaderLite(B=8, T=1024)
# step 49, loss: 5.99915075302124, dt: 302.22ms, tok/sec: 27106.457056528754  (19 Gb memory on GPU)
# DataLoaderLite(B=16, T=1024)
# torch.OutOfMemoryError: CUDA out of memory. Tried to allocate 1.54 GiB. GPU 0 has a total capacity of 23.70 GiB of which 1.26 GiB is free. Including non-PyTorch memory, this process has 22.19 GiB memory in use. Of the allocated memory 21.03 GiB is allocated by PyTorch, and 26.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


# andrej stats 
# step 49, loss: 6.06, dt: 199.64ms, tok/sec: 54678.65

---
# torch.complie

---

- It is bascially a complier for Neural Networks. 
- Like GCC for C/C++ code 
- https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html


In [ ]:
# you can use it just with one line of code 
# torch.compile(model)

# this line of code will cost you compliation time. 
# but it will make your code run a lot faster. 

In [1]:
# https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&Distribution=Ubuntu&target_version=22.04&target_type=deb_network

In [ ]:
# ** Using torch.compile**
# have to udpate the nvidia drivers
# step 49, loss: 5.977720737457275, dt: 180.91ms, tok/sec: 45281.44075149841


# Andrej stats 
# step 49, loss: 6.05, dt: 129.66, tok/sec: 126364.38


- Unless, you are debugging adn  you want your code to run really fast. 
- you should always use torch.compile(model)
- Speedup mainly comes from reducing Python overhead and GPU read/writes.


**So what is happending under the hood**
- In code of GPT2 class, all we have is algorithmic description of what we would like to happen in our network and torch complie will analize the entire thing.
- And, with the benefit of knowing, exactly what doing to happen it doesn't have to run a egear mode, it doesn't have to go like layer by layer and it optimize the process. 
- it will take out the Python interpreter from the forward pass entirely and will complie the entire NN as single object with no python interpreter involved. 


**Understanding GPU read/Writes**

- How a this operation works? 

say you have (x ** 3) ** 0.5 where x is tensor of size (10, 20)

- the tensor will be stored in GPU memeory and when you execute this operation. This tensor will travel to GPU Cores, performes the operation and the result will be saved back to the GPU memory. 
- During, this travel of tensor backandforth from memory to core the GPU's memory bandwith comes into picture. 
- not for ** 0.5  this operation GPu will repeat the process of moving the tensor from memory to cores, do the operation and write back to it. 
- with torch.complie() it already know how to what are operation you are going to perform on the tensor and can load the tensor once, do all the operation and write back the result. This saved al the round trips. 
- This is one of example which is called as kernel fusion. 
- Look at the video at 1:55:00 for more deeper understanding of GPU architure. 

---
# Flash attention
---

- It come sfrom the papaer: Flash attention: Fast and memory-efficient exact attention with IO-Awareness
- 

In [ ]:
# flash attention implement this 4 lines very very quickly

    # att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
    # att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
    # att = F.softmax(att, dim=-1)
    # y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

# How does it do that??

- Flash attention is a kenerl Fusion operation 
- Look at the Fig1 from the paper, the left side of the picture is equal to the above 4 lines that we have (except for the dropout, as we are not using it).
- Instead of using these 4 lines we are fusing them into a single fused kernel of flash attention. 
- It's a kernel fusion algorigthm but it's a kernel fusion that torch compile can't find and the reason it can't find it because it required a algorithmic re-write of how attenstion is actually implemented here in this case. 
- If you count the num of flops, the flash attention does more flops than the attention is used using those 4 lines of code. But, it is significianlty faster, infact 7.6 times faster. 
- That is because, it is very mindful of the memory hierarchy. It is very mindful of what is in HBM (High Bandwidth Memory), what's in the shared memory, and it's very careful about how to orchestrates the computation such that we have fewer read and writes to HBM and so even though we are doing more flops, the expensive part is they load and store into hbm and that's what they avoid. 
- Flash attention designed that self-attention (att matix) never get materialized at any point and never gets read or written to the HBM.
- And, the way that this is acheivided. The fundamental algorithmic rewrite here relies on this online softmax trick.
- Online Softmax trick (come from a previous paper). It shows how you can incrementally evaluate a softmax without having to sort of realize all of the inputs of the softmax to do the normalization.And, you do that by having these intermediate variable m and l (looking at equation from the paper 3.1 section)
and there is a update to them that allows you to evaluate the softmax in online manner. 
- Recently; Flash Attention-2 : Faster attention with better parallelism and work partitioning that has additional gains to how it calculates flash attention and the original paper that this is based on basically "Online normalizer calculation for softmax" by Nvidia. 
- Key take away from these papers
    - Be aware of memory hierarchy. 
    - The fact that flops don't matter and the entire memory access pattern matters.
    - torch complie is amazing but there are many optimization still avaiable to us that potentially torch complie can't find. 


In [ ]:
# after implementing Flash-attention. 
# step 49, loss: 5.978326797485352, dt: 147.64ms, tok/sec: 55486.14267927764

# andrej stats 
# step 49, loss: 6.05, dt: 96.64ms, tok/sec: 169723.40


---
# Nice/Ugly numbers
---

- Nice numbers: which have a lot of power of 2 
- Ugly numbers : 13, 17 , 7 
- You always want to use Nice numbers in all of your code that deals with NN or Cuda. Because everything in Cuda works in like power of 2 and a lot of kernels are written in power of 2 and there are a lot of blocks of size 16, 64 and so on. 
- So, go over the entire code and look for ugly numbers 
- vocab_size = 50257 is very ugly number.
- How to fix this?
- you basically, increase the number until it's nearest power of 2 that you like. 
- 50304 : as it can be divided by 8, 16, 32, 64, 128. This is like we are adding fake tokens and these tokens are never used. 
- If think about it by adding these fake token, at the last layer i.e the classifier what this is doing is that we are predicting additional dimensions at the classifier now and we are predicting probalilites for the tokens that of course never be present in the training set. Therefore, the network has to learn that these probabalities have to driven to zero. So, the logits that network produces have to drive those dimensions of output to -inf. 
- In our shakespeare example, anyway we just have few thousand tokens and the rest of them is just pushed down to zero. we are just adding few more. 
- We have to make sure that this doesn't break anything.
- In conclusion, if you think about it we are adding computation but we are running faster and the reason for this as we have discuss above. 

